In [ ]:
from typing import List, Optional
from typing_extensions import TypedDict

# 장소 정보를 담는 TypedDict 정의
class KakaoPlace(TypedDict):
    name: str
    address: str
    url: str

# 대화 상태를 담는 TypedDict 정의
class State(TypedDict):
    messages: List  # 실제 타입은 아래에서 Annotated를 통해 정의 예정
    search_query: Optional[str]
    search_results: Optional[List[KakaoPlace]]

In [ ]:
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from typing import Annotated
class State(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    search_query: Optional[str]
    search_results: Optional[List[KakaoPlace]]

In [ ]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# 키워드 추출 도구 함수 정의
@tool
def extract_keyword(user_request: str) -> str:
    """사용자 요청에서 장소 검색을 위한 짧고 명확한 키워드를 추출합니다."""
    prompt = (
        "다음 요청에서 장소 검색에 사용할 키워드를 한 문장으로 추출해줘.\n"
        "- 키워드는 띄어쓰기로 구분된 한 문장이어야 해.\n"
        "- 다른 설명은 하지 말고 키워드만 알려줘.\n\n"
        f"요청: '{user_request}'"
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    keyword = response.content.strip().strip('"').strip("'")
    return keyword

In [ ]:
import requests
@tool
def kakao_place_search(query: str) -> list[KakaoPlace]:
    """카카오 장소 검색 API를 사용하여 장소를 검색합니다."""
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {os.getenv('KAKAO_API_KEY')}"}
    params = {"query": query, "size": 5}
    response = requests.get(url, headers=headers, params=params)
    data = response.json().get("documents", [])
    results: list[KakaoPlace] = []
    for place in data:
        results.append({
            "name": place.get("place_name", ""),
            "address": place.get("address_name", ""),
            "url": place.get("place_url", "")
        })
    return results

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

def chatbot_node(state: State):
    ai_response = llm_with_tools.invoke(state["messages"])
    return {"messages": [ai_response]}

tools = [extract_keyword, kakao_place_search]
llm_with_tools = llm.bind_tools(tools)
tool_node = ToolNode(tools)
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

In [ ]:
from langchain_core.messages import HumanMessage
# 초기 상태 설정: 사용자 질문 입력
initial_state: State = {
    "messages": [HumanMessage(content="대전을 대표하는 빵집은?")],
    "search_query": None,
    "search_results": None
}
# 그래프 실행하여 결과 얻기
final_state = graph.invoke(initial_state)

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage
for msg in final_state["messages"]:
    if isinstance(msg, HumanMessage):
        print("👤 사용자:", msg.content)
    elif isinstance(msg, AIMessage):
        print("🤖 챗봇:", msg.content)
        if msg.tool_calls:
            for call in msg.tool_calls:
                print(f"  ↪ (도구 요청: {call['name']} {call['args']})")
    elif isinstance(msg, ToolMessage):
        print(f"🔧 도구[{msg.name}] 결과:", msg.content)

In [ ]:
# 추가 테스트 예시
initial_state["messages"] = [HumanMessage(content="서울에서 애견동반 가능한 카페 알려줘")]
final_state = graph.invoke(initial_state)
for msg in final_state["messages"]:
    if isinstance(msg, HumanMessage):
        print("👤 사용자:", msg.content)
    elif isinstance(msg, AIMessage):
        print("🤖 챗봇:", msg.content)
        if msg.tool_calls:
            for call in msg.tool_calls:
                print(f"  ↪ (도구 요청: {call['name']} {call['args']})")
    elif isinstance(msg, ToolMessage):
        print(f"🔧 도구[{msg.name}] 결과:", msg.content)